In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
from sklearn import metrics
from numpy import mean
from numpy import std
from mlxtend.evaluate import paired_ttest_5x2cv,combined_ftest_5x2cv





from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, roc_auc_score,precision_score
from sklearn.model_selection import StratifiedKFold 
# evaluate label propagation on the semi-supervised learning dataset

dataSet = sp.loadmat('breastw.mat')
X = dataSet['X']
y = dataSet['y']
Covariance=['spherical', 'tied', 'diag', 'full']
Params=['kmeans', 'random']
skf = StratifiedKFold(n_splits=2) 
fold=0
test=False
for train_index, test_index in skf.split(X, y):
        fold=fold+1
        k = 0
        liste =[0]*8
        
        X_train, X_test = X[train_index,:], X[test_index,:]
        y_train, y_test = y[train_index], y[test_index]
        for i in Covariance:
                for j in Params:
                       
                       
                        gmm = GaussianMixture(n_components=2,init_params=j,covariance_type=i)
                        gmm.fit(X_train,y_train)
                        y_pred=gmm.predict(X_test)
                        if precision_score(y_test,y_pred)< 0.1:
    
                                y_pred = np.array([0 if label == 1 else 1 for label in y_pred])
                        
                        F1=round(f1_score(y_test,y_pred),3)
                        recall=round(recall_score(y_test,y_pred),3)
                        precision=round(precision_score(y_test,y_pred,zero_division=0),3)
                        accuracy=round(accuracy_score(y_test,y_pred),3)
                        fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred)
                        roc_auc = metrics.auc(fpr, tpr)
                        if test==False :
                                data = np.array([[i,j,F1,recall,precision,accuracy,roc_auc,fold]])
                                test=True
                        else :
                                data = np.append(data, [[i,j,F1,recall,precision,accuracy,roc_auc,fold]],axis=0)
                        liste[k]=gmm
                        k = k+1
                       
        for estimator1, estimator2 in zip(liste,liste[1:]):
                t, p = paired_ttest_5x2cv(estimator1= estimator1, estimator2=estimator2,X=X, y=y, scoring='f1', random_seed=None)
        
                print('P-value: %.3f, t-Statistic: %.3f' % (p, t))

              
                        
             


P-value: 1.000, t-Statistic: 0.000
P-value: 0.989, t-Statistic: 0.014
P-value: 0.595, t-Statistic: -0.567
P-value: 0.255, t-Statistic: 1.285
P-value: 0.026, t-Statistic: 3.126
P-value: 0.310, t-Statistic: -1.130
P-value: 1.000, t-Statistic: 0.000
P-value: 1.000, t-Statistic: 0.000
P-value: 0.351, t-Statistic: 1.028
P-value: 0.339, t-Statistic: -1.057
P-value: 0.196, t-Statistic: -1.491
P-value: 0.396, t-Statistic: 0.929
P-value: 0.275, t-Statistic: -1.227
P-value: 0.933, t-Statistic: 0.089


In [106]:
import pandas as pd
from scipy.stats import mannwhitneyu
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = pd.DataFrame(data=data,columns=['Covariance','Parametre','F1','recall','precision','accuracy','AUC','numero_fold'])
df
mod = ols('C(Covariance)~ F1',data=df).fit()
results = sm.stats.anova_ml(mod,type = 2)
results


ValueError: endog has evaluated to an array with multiple columns that has shape (16, 4). This occurs when the variable converted to endog is non-numeric (e.g., bool or str).

In [ ]:
df.to_csv ("test.csv", sep= ",",na_rep= "",float_format= None,columns= None,header= True,index= True,index_label= None, mode= "w", encoding= None,compression= "infer",quoting= None,line_terminator= None,chunksize= None,date_format= None,doublequote= True,escapechar= None,decimal= ".")